# Modules

In [1]:
%matplotlib widget
import os
import glob
import psfex
import pickle
import sncosmo
import numpy as np
import pandas as pd
import iminuit as im
import astropy as ap
import ipywidgets as ipw
import matplotlib.pyplot as plt
import ztfquery
import ztfimg
import ztflc

from ztfimg import dao
from ztfimg import image
from ztflc import fitter
from ztflc import diffdata
from astropy.io import fits
from ztfquery import marshal
from astrobject import photometry
from ztflc import forcephotometry
from scipy.stats import multivariate_normal
from sncosmo.models import Source, Model, get_source
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sncosmo import get_bandpass, get_magsystem
from sncosmo.photdata import photometric_data

## Img loading

In [2]:
root = "../Data/"
sciimg = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimg.fits"
mask = root + "ztf_20190124095417_000403_zr_c01_o_q1_mskimg.fits"
psFranck = root + "ztf_20190124095417_000403_zr_c01_o_q1_psfcat.fits"
psfcent = root + "ztf_20190124095417_000403_zr_c01_o_q1_sciimgdaopsfcent.fits"

hdul = fits.open(sciimg)
pdul = fits.open(psFranck)
cdul = fits.open(psfcent)

cdata = cdul[0].data
header = hdul[0].header

psfcat = pd.DataFrame(pdul[1].data)
x_list, y_list = psfcat['xpos'], psfcat['ypos']

z = image.ScienceImage(sciimg, mask)
z.load_source_background()
z.set_catalog(psfcat, 'psfcat')

row = z.header["NAXIS1"]
col = z.header["NAXIS2"]

## Create sciunc

FILE_OUT = sciimg[:-11] + 'sciunc.fits'

fake_sci = fits.HDUList(fits.PrimaryHDU(data=z.sourcebackground.rms(),
                                        header=header))
fake_sci.writeto(FILE_OUT, overwrite=True)

## Show img+cat

plt.close()
ax = z.show('dataclean')
ax.scatter(x_list, y_list, color='C1', marker='+')

## Show PSF stamps at center

## PSF choice?

In [3]:
subdir_list = [subdir_name for subdir_name in os.listdir('../Data/config_ZOGY/')]

psfchoice = ipw.Dropdown(options=subdir_list,
                         description='Which psfex.config?',
                         value=subdir_list[1])
def choose_psf(psfdir):
    global psfimg, psfimg_cent, psfimg_smpl, psfobj
    psfilepath = "../Data/config_ZOGY/" + psfdir + '/test.psf'
    psfobj = psfex.PSFEx(psfilepath)
    psfimg = psfobj.get_rec(row, col)
    psfimg_cent = psfobj.get_rec(row/2, col/2)
    psfimg_smpl = psfobj.get_rec(100, 3050)

int_choose_psf = ipw.interactive(choose_psf, psfdir=psfchoice)
display(int_choose_psf)

interactive(children=(Dropdown(description='Which psfex.config?', index=1, options=('psfex_wrapper', 'BlackGEM…

### Testing `psfex_out.cat`

In [4]:
psfoutcatpath = root + 'config_ZOGY/' + psfchoice.value + '/psfex_out.cat'
#odul = fits.open(psfoutcatpath)
#testcat = pd.DataFrame(tdul[2].data)

## Norm it on daocent

In [5]:
daocent_max = np.max(np.ravel(cdata))

In [6]:
psfimg_norm = np.sum(np.ravel(psfimg))
psfimg_normed = psfimg/psfimg_norm
psfimg_normed_max = np.max(psfimg_normed)
psfimg_scaled = psfimg_normed*daocent_max/psfimg_normed_max

In [7]:
plt.close()
fig = plt.figure(figsize=[7, 3])

ax = fig.add_subplot(131)
ax.imshow(psfimg_scaled,
          origin='lower', interpolation='nearest')
plt.title('PSF for ' + psfchoice.value)

ax = fig.add_subplot(132)
ax.imshow(cdata,
          origin='lower', interpolation='nearest')
plt.title('sciimgdaopsfcent')

ax = fig.add_subplot(133)
im = ax.imshow((psfimg_scaled-cdata)/np.max(cdata),
               origin='lower', interpolation='nearest')
plt.title('Diff/max_middle')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig.savefig('../Images/' + 'pdf' + '/PSFEx-out_vs_psfcent_' + psfchoice.value + '.pdf')
fig.savefig('../Images/' + 'png' + '/PSFEx-out_vs_psfcent_' + psfchoice.value + '.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Norm it on psfcent

In [8]:
psfcent_max = np.max(np.ravel(psfimg_cent))
psfimg_norm = np.sum(np.ravel(psfimg_smpl))
psfimg_normed = psfimg_smpl/psfimg_norm
psfimg_normed_max = np.max(psfimg_normed)
psfimg_scaled = psfimg_normed*psfcent_max/psfimg_normed_max

In [9]:
plt.close()
fig = plt.figure(figsize=[7, 3])

ax = fig.add_subplot(131)
ax.imshow(psfimg_scaled)
plt.title('PSf @ (100, 3050)')

ax = fig.add_subplot(132)
ax.imshow(psfimg_cent)
plt.title('PSF @ (1536, 1540)')

ax = fig.add_subplot(133)
im = ax.imshow((psfimg_scaled-psfimg_cent)/np.max(psfimg_cent))
plt.title('Diff/max_middle')

divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(im, cax=cax, orientation='vertical')

fig.suptitle('PSF for ' + psfchoice.value, fontsize=16)

fig.savefig('../Images/' + 'png' + 'psfsmpl_psfcent_diff_' + psfchoice.value + '.png',
            bbox_inches='tight')
fig.savefig('../Images/' + 'pdf' + 'psfsmpl_psfcent_diff_' + psfchoice.value + '.pdf',
            bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …